In [4]:
!pip install snowflake-snowpark-python

                                              0.0/331.0 kB ? eta -:--:--
     ------------                           112.6/331.0 kB 3.3 MB/s eta 0:00:01
     -------------------------------------  327.7/331.0 kB 3.4 MB/s eta 0:00:01
     -------------------------------------- 331.0/331.0 kB 2.9 MB/s eta 0:00:00
                                              0.0/877.1 kB ? eta -:--:--
     ------                                 153.6/877.1 kB 3.1 MB/s eta 0:00:01
     ------------------                     430.1/877.1 kB 4.5 MB/s eta 0:00:01
     ---------------------------            624.6/877.1 kB 4.4 MB/s eta 0:00:01
     -------------------------------------- 877.1/877.1 kB 5.0 MB/s eta 0:00:00
                                              0.0/105.0 kB ? eta -:--:--
     -------------------------------------- 105.0/105.0 kB 5.9 MB/s eta 0:00:00
  Using cached platformdirs-3.11.0-py3-none-any.whl (17 kB)
  Attempting uninstall: platformdirs
    Found existing installation: platformdir

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
black 0.0 requires click>=8.0.0, but you have click 7.1.2 which is incompatible.
python-lsp-black 1.2.1 requires black>=22.3.0, but you have black 0.0 which is incompatible.


In [11]:
%%writefile testmapkoreng.py

import json
import streamlit as st
import pandas as pd
import folium
from streamlit_folium import folium_static
import requests
from datetime import date, datetime
import plotly.express as px
import numpy as np
from typing import Iterable
from snowflake.snowpark.session import Session

# 데이터 로딩
data = pd.read_excel("streamlit_data.xlsx")

class OurFilter:
    def __init__(self, human_name, table_column, widget_id, widget_type):
        self.human_name = human_name
        self.table_column = table_column
        self.widget_id = widget_id
        self.widget_type = widget_type
        self.enabled = False  # Initialize enabled attribute
        self.value = None  # Initialize value attribute

    def enable(self):
        self.enabled = True

    def create_widget(self):
        if self.widget_type == st.checkbox:
            self.value = st.checkbox(self.human_name, key=self.widget_id)
        elif self.widget_type == st.date_input:
            self.value = st.date_input(self.human_name, key=self.widget_id)
        elif self.widget_type == st.multiselect:
            options = sorted(map(str, data[self.table_column].unique()))
            # Remove the default value from here
            self.value = st.multiselect(self.human_name, options, key=self.widget_id)
    

# Define a custom SessionState class
class SessionState:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

# Initialize session state with default values
if 'start_date_widget' not in st.session_state:
    st.session_state.start_date_widget = data['일자'].min()

if 'end_date_widget' not in st.session_state:
    st.session_state.end_date_widget = data['일자'].max()

if 'topics_widget' not in st.session_state:
    st.session_state.topics_widget = data['topics'].unique().tolist()

def init_connection():
    # Placeholder for initializing connection
    return None

def _get_human_filter_names(filters):
    return [f.human_name for f in filters]

def _is_any_filter_enabled():
    return any(f.enabled for f in st.session_state.filters)

def _get_active_filters():
    return [f for f in st.session_state.filters if f.enabled]

def draw_sidebar():
    """Should include dynamically generated filters"""

    with st.sidebar:
        selected_filters = st.multiselect(
            "Select which filters to enable",
            list(_get_human_filter_names(st.session_state.filters)),
            [],
        )
        for _f in st.session_state.filters:
            if _f.human_name in selected_filters:
                _f.enable()

        if _is_any_filter_enabled():
            with st.form(key="input_form"):

                for _f in _get_active_filters():
                    _f.create_widget()
                st.session_state.clicked = st.form_submit_button(label="Submit")
        else:
            st.write("Please enable a filter")

if __name__ == "__main__":
    # Initialize the filters
    session = init_connection()
    OurFilter.session = session
    MY_TABLE = "news_data"
    OurFilter.table_name = MY_TABLE

    # Dynamically generate filters based on selected columns
    st.session_state.filters = [
        OurFilter(
            human_name="Start date",
            table_column="일자",
            widget_id="start_date_widget",
            widget_type=st.date_input,
        ),
        OurFilter(
            human_name="End date",
            table_column="일자",
            widget_id="end_date_widget",
            widget_type=st.date_input,
        ),
        OurFilter(
            human_name="Choose topics",
            table_column="topics",
            widget_id="topics_widget",
            widget_type=st.multiselect,
        )
    ]
    # # Explicitly initialize session state values
    # st.session_state.start_date_widget = data['일자'].min()
    # st.session_state.end_date_widget = data['일자'].max()
    # st.session_state.topics_widget = ['전쟁']
    
    draw_sidebar()
    
     # Access the selected values
    start_date_widget = st.session_state.start_date_widget
    end_date_widget = st.session_state.end_date_widget
    topics_widget = st.session_state.topics_widget
    
    # 한국 도시와 외국 도시 데이터를 나누어 저장
    korean_data = data[data['외국유무'] == '한국']
    foreign_data = data[data['외국유무'] == '외국']
    
    
    # tabs 만들기
    tab1, tab2 = st.tabs(["국내", "해외"])

    # tab1 내용물 구성하기
    with tab1:
        # 제목 넣기
        st.markdown("## 국내 재난/재해 상황")

        def draw_korea_map(start, end, topic):
            global_distriction_jsonurl = 'https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-provinces-2018-geo.json'
            global_dict = json.loads(requests.get(global_distriction_jsonurl).text)

            # Convert start_date and end_date to datetime64[ns]
            start_date = np.datetime64(start)
            end_date = np.datetime64(end)

            return_df = data[data["topics"].isin(topic)]
            return_df = return_df.loc[return_df.일자.between(start_date, end_date), :]
            return_df.reset_index(drop=True, inplace=True)
            return_df = return_df.groupby(["city", "외국유무"], as_index=False)[["risk"]].sum()

            m1 = folium.Map(location=[36, 128], tiles='cartodbpositron', zoom_start=7, scrollWheelZoom=False)
            korea_result = return_df.loc[return_df["외국유무"] == "한국", :]
            choro1 = folium.Choropleth(
                data=korea_result,
                geo_data=global_dict,
                columns=["city", "risk"],
                key_on="feature.properties.name",
                mapbox_style="carto-positron",
                fill_color="YlOrRd"
            )

            choro1.add_to(m1)
            st_data = folium_static(m1, width=1000)

        # 사용자 입력을 기반으로 지도 그리기
        draw_korea_map(st.session_state.start_date_widget, st.session_state.end_date_widget, st.session_state.topics_widget)

        col1, col2 = st.columns([5, 5])

        with col1:
            topics_distribution = korean_data['topics'].value_counts().reset_index()
            fig_topics = px.pie(
                topics_distribution,
                names='index',
                values='topics',
                title='한국 Topics 분포',
                hole=0.4,  # 도넛 차트 설정
                color_discrete_sequence=px.colors.qualitative.Set3  # 색상 조정
            )
            fig_topics.update_layout(
                width=400,
                height=400,
                title_x=0.5,
                margin=dict(l=0, r=0, t=30, b=0),
                showlegend=True,  # 범례 표시
                legend_title="Topics" 
            )
            st.plotly_chart(fig_topics)

    # 가운데 열에는 한국 도시별 리스크 그래프 표시
        with col2:
            korean_top_5 = korean_data.groupby('city')['risk'].sum().nlargest(5).reset_index()
            fig_korean = px.bar(
                korean_top_5,
                y='risk',
                x='city',
                orientation='v',
                title="Top 5 한국 도시별 리스크",
                color='risk',  # 색상을 리스크에 따라 설정
                color_continuous_scale='viridis'  # 색상 맵 설정
            )
            fig_korean.update_layout(
                width=400,
                height=400,
                title_x=0.5,
                margin=dict(l=0, r=0, t=30, b=0),
                showlegend=True
            )
            st.plotly_chart(fig_korean)


    # col1, col2 = st.columns([70, 30])
    # with col1:
    #     return_df = data[data["topics"].isin(topics)]
    #     return_df = return_df.loc[return_df.일자.between(start_date, end_date), :]
    #     return_df.reset_index(drop=True, inplace=True)
    #     return_df = return_df.groupby(["city", "외국유무"], as_index=False)[["risk"]].sum()
        # exec("a1 =return_df.groupby('city')['risk'].sum().idxmax()")
        # exec("a2 = return_df.groupby('city')['risk'].sum().nlargest(2).index[1]")
        # st.radio("가장 위험도가 높은 나라",[a1, a2])
        # st.dataframe(filtered_data[filtered_data['외국유무'] == '한국'][['일자', 'city','제목', '요약']])


    # tab2 내용물 구성하기
    with tab2:
        # 제목 넣기
        st.markdown("## 해외 재난/재해 상황")

        # draw_map 함수 정의
        def draw_foreign_map(start, end, topic):
            with open('국가별위도경도.json', 'r') as json_file:
                global_dict = json.load(json_file)

            return_df = data[data["topics"].isin(topic)]
            return_df = return_df.loc[return_df.일자.between(start, end), :]
            return_df.reset_index(drop=True, inplace=True)
            return_df = return_df.groupby(["city", "외국유무"], as_index=False)[["risk"]].sum()

            m2 = folium.Map(location=[31.768319, 35.21371], tiles='cartodbpositron', zoom_start=7, scrollWheelZoom=True)
            nation_result = return_df.loc[return_df["외국유무"] == "외국", :]
            choro2 = folium.Choropleth(
                data=nation_result,
                geo_data=global_dict,
                columns=["city", "risk"],
                key_on="feature.properties.name",
                mapbox_style="carto-positron",
                fill_color="YlOrRd"
            ).add_to(m2)

            choro2.add_to(m2)
            st_data = folium_static(m2, width=1000)

        draw_foreign_map(st.session_state.start_date_widget, st.session_state.end_date_widget, st.session_state.topics_widget)



            # 컬럼 설정
        col1, col2 = st.columns([5, 5])

        with col1:
            topics_distribution = foreign_data['topics'].value_counts().reset_index()
            fig_topics = px.pie(
                topics_distribution,
                names='index',
                values='topics',
                title='외국 Topics 분포',
                hole=0.4,  # 도넛 차트 설정
                color_discrete_sequence=px.colors.qualitative.Set3  # 색상 조정
            )
            fig_topics.update_layout(
                width=400,
                height=400,
                title_x=0.5,
                margin=dict(l=0, r=0, t=30, b=0),
                showlegend=True,  # 범례 표시
                legend_title="Topics" 
            )
            st.plotly_chart(fig_topics)

        with col2:
            foreign_top_5 = foreign_data.groupby('city')['risk'].sum().nlargest(5).reset_index()
            fig_foreign = px.bar(
                foreign_top_5,
                y='risk',
                x='city',
                orientation='v',
                title="Top 5 외국 도시별 리스크",
                color='risk',
                color_continuous_scale='viridis'
            )
            fig_foreign.update_layout(
                width=400,
                height=400,
                title_x=0.5,
                margin=dict(l=0, r=0, t=30, b=0),
                showlegend=True
            )
            st.plotly_chart(fig_foreign)

        # col1, col2 = st.columns([70, 30])
        # with col1:
        #     st.dataframe(filtered_data[filtered_data['외국유무'] == '외국'][['일자', 'city','제목', '요약']])


Overwriting testmapkoreng.py
